In [13]:
import numpy as np
from PIL import Image, ImageStat
import os

path = '/data3/ian/dsmil-wsi/WSI/C16_training/single'
dirs = os.listdir(path)
mean_list = []
std_list = []
for d in dirs:
    print(d)
    d = os.path.join(path,d)
    img_list = os.listdir(d)
    img_mean = []
    img_std = []
    for img in img_list:
        print(img)
        img = os.path.join(d, img)
        patches = os.listdir(img)
        patch_mean = []
        patch_std = []
        for p in patches:
            #print(patches)
            patch = os.path.join(img, p)
            patch = Image.open(patch)
            patch = np.array(patch)/255
            stat = ImageStat.Stat(patch)
            patch_mean.append(stat.mean)
            patch_std.append(stat.std)
            
        tmp = np.mean(patch_mean)
        img_mean.append(tmp)

        tmp = np.mean(patch_std)
        img_std.append(tmp)
         
    tmp = np.mean(img_mean)
    mean_list.append(tmp)
    tmp = np.mean(img_std)
    std_list.append(tmp)
    
print(np.mean(mean_list))
print(np.mean(std_list))

    


1
tumor_097


TypeError: first argument must be image or list

In [9]:
path = '/data3/ian/dsmil-wsi/WSI/C16_training/single/1/tumor_001/149_271.jpeg'
patch = Image.open(path)
stat = ImageStat.Stat(patch)
print(stat.mean)

[143.3711535395408, 92.78948102678571, 123.36892139668367]


In [14]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [15]:
root = '/data3/ian/dsmil-wsi/WSI/project_tiff/single/1'

train_dataset = datasets.ImageFolder(root, transform=transforms.ToTensor())
#train_dataset = datasets.CIFAR10(root='dataset/', train=True, 
#                                 transform=transforms.ToTensor(), download=True)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=64)

In [18]:
print(np.shape(train_dataset.imgs))

(3434827, 2)


In [19]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [20]:
m, s = get_mean_and_std(train_dataloader)

In [21]:
print(m,s)

tensor([0.8865, 0.5624, 0.8367]) tensor([0.1441, 0.2491, 0.1156])


In [22]:
root = '/data3/ian/dsmil-wsi/WSI/project_tiff/single/0'

train_dataset = datasets.ImageFolder(root, transform=transforms.ToTensor())
#train_dataset = datasets.CIFAR10(root='dataset/', train=True, 
#                                 transform=transforms.ToTensor(), download=True)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=64)

In [23]:
m, s = get_mean_and_std(train_dataloader)

In [24]:
print(m,s)

tensor([0.9219, 0.6062, 0.8495]) tensor([0.1165, 0.2028, 0.1018])


In [ ]:
import torch
from tqdm import tqdm
from torchvision.datasets.cifar import CIFAR100
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

transform = transforms.Compose(
    [
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]
)
data_set = CIFAR100('./data/cifar', train=True, transform=transform, download=True)
data_loader = DataLoader(data_set, batch_size=24, num_workers=8, shuffle=False)

nb_samples = 0.
channel_mean = torch.zeros(3)
channel_std = torch.zeros(3)
for images, targets in tqdm(data_loader):
    # scale image to be between 0 and 1
    N, C, H, W = images.shape[:4]
    data = images.view(N, C, -1)

    channel_mean += data.mean(2).sum(0)
    channel_std += data.std(2).sum(0)
    nb_samples += N

channel_mean /= nb_samples
channel_std /= nb_samples
print(channel_mean, channel_std)

In [26]:
import os
import numpy as np
import shutil

img_path = '/data3/ian/dsmil-wsi/WSI/project_tiff/single/0'
store_path = '/data3/ian/dsmil-wsi/WSI/project_tiff/train/0'
dir_list = os.listdir(img_path)
for d in dir_list:
    dir_path = os.path.join(img_path, d)
    files = os.listdir(dir_path)
    for f in files:
        file = os.path.join(dir_path,f)
        des = os.path.join(store_path,d+'_'+f)
        shutil.copyfile(file, des)
        